In [ ]:
#default_exp lenet

In [ ]:
# export
from pwc_gabor_layer.core import GaborLayer, SigmaRegularizer
from pwc_gabor_layer.alexnet import GaborBlock
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
# hide
%load_ext autoreload
%autoreload 2

# Implementing LeNet in Keras

In [ ]:
# exports
class LeNet(keras.Model):
    def __init__(self, num_classes=10, input_channels=1, 
                kernels1=None, kernels2=None,
                learn_orientations=False, **kwargs):
        super().__init__(*kwargs)
        
        self.learn_orientations = learn_orientations
        self.num_classes = num_classes
        self.input_channels = input_channels
        
        self.conv1 = self.add_conv_layer(kernels1, 6, (5, 5), strides=1)
        self.conv2 = self.add_conv_layer(kernels2, 16, (5, 5), strides=1)
            

        # Define output (dense connections)
        self.linear1 = keras.layers.Dense(256, activation='relu')
        self.linear2 = keras.layers.Dense(256, activation='relu')
        self.linear3 = keras.layers.Dense(self.num_classes, activation='sigmoid')
        
        # Supporting layers
        self.maxpool = keras.layers.MaxPooling2D()
        self.flatten = keras.layers.Flatten()
            
    def call(self, x, training=False):
        # block 1
        x = self.conv1(x)
        x = self.maxpool(x)
        # block 2
        x = self.conv2(x)
        x = self.maxpool(x)
        # classifier
        x = self.flatten(x)
        
        x = self.linear1(x)
        x = self.linear2(x)
        x = self.linear3(x)
        
        return x
    
    
    def add_conv_layer(self, gabor, filters_conv, kernel_size, strides):
        
        if gabor:
            block = GaborBlock(filters_gabor=gabor,
                                    filters_conv=filters_conv,
                                    kernel_size=kernel_size,
                                    strides=strides,
                                    learn_orientations=self.learn_orientations)
        else:
            block = keras.layers.Conv2D(filters=filters_conv,
                                       kernel_size=kernel_size,
                                       strides=strides,
                                       activation='relu',
                                       padding='same')
        return block


In [ ]:
# hide
# Create some test examples, just to be sure
img = np.ones((1, 28, 28, 2)).astype(np.float32)
y = np.zeros((1, 10), dtype=int)
y[0, 0] = 1
# Test Lenet with gabor layers
LNG = LeNet(kernels1=2, kernels2=2)
out = np.array(LNG(img))

LNG.compile(optimizer='sgd', loss='categorical_crossentropy')
LNG.fit(x=img, y=y)

1/1 [==============================] - 0s 0s/step - loss: 1.9185


In [ ]:
# hide
# Test alexnet with conv layer
LNC = LeNet()
out = np.array(LNC(img))

LNC.compile(optimizer='sgd', loss='categorical_crossentropy')
LNC.fit(x=img, y=y)

1/1 [==============================] - 0s 0s/step - loss: 2.3705
